# Demo: using NotionChatBot

In [1]:
%load_ext autoreload
%autoreload 2

load `.env` to environment variables, used for LangSmith

In [2]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

Load configuration file

In [3]:
import tomllib

with open('../.config.toml', 'rb') as f:
    _CONFIGS = tomllib.load(f)

initialize llm and tokenizer

In [4]:
from langchain_community.llms import LlamaCpp

llm = LlamaCpp(
    model_path=_CONFIGS['model_path']+'/'+_CONFIGS['model_mapping'][_CONFIGS['model_name']],
    name=_CONFIGS['model_name'], 
    **_CONFIGS['llm']
)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    _CONFIGS['model_name'], 
    trust_remote_code=True
)

/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! conversation is not default parameter.
                conversation was transferred to model_kwargs.
                Please confirm that conversation is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /Users/fred/Documents/models/zephyr-7b-beta.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-beta
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096


initialize NotionChatBot

In [5]:
from my_notion_companion.notion_chatbot import NotionChatBot

c = NotionChatBot(llm, tokenizer, '../.config.toml', verbose=True)

2024-03-13 16:37:04.765 | INFO     | my_notion_companion.utils:load_notion_documents:24 - Load data from existing offline copy.
2024-03-13 16:37:04.802 | INFO     | my_notion_companion.document_metadata_filter:__init__:18 - Setting metadata fuzzy match threshold to: 0.8.
2024-03-13 16:37:04.802 | INFO     | my_notion_companion.query_analyzer:__init__:47 - Initialize Query Analyzer.


Invoke the chatbot. The first query will be used to search in lexical and semantic databases to retrieve relevant documents. The chatbot will then answer the question based on the retrieved docuemtns.

In [6]:
c.invoke("谁曾在步行者队效力？从“写作”中找答案。")

2024-03-13 16:37:04.857 | INFO     | my_notion_companion.notion_chatbot:invoke:51 - Try lexical search.

llama_print_timings:        load time =    8864.86 ms
llama_print_timings:      sample time =       2.31 ms /    24 runs   (    0.10 ms per token, 10376.13 tokens per second)
llama_print_timings: prompt eval time =    8863.30 ms /   759 tokens (   11.68 ms per token,    85.63 tokens per second)
llama_print_timings:        eval time =     809.82 ms /    23 runs   (   35.21 ms per token,    28.40 tokens per second)
llama_print_timings:       total time =    9716.46 ms /   782 tokens
2024-03-13 16:37:14.751 | INFO     | my_notion_companion.query_analyzer:clean_output:51 - Query Analyzer output: 关键词：谁曾在步行者队效力|搜索范围：写作
2024-03-13 16:37:14.755 | INFO     | my_notion_companion.query_analyzer:parse_output:73 - 
Query Analyzer output
keyword: ['谁曾在步行者队效力']
search domains:['写作']
2024-03-13 16:37:14.756 | INFO     | my_notion_companion.retriever:_filter_documents:110 - filter found by query ana

'文档1 提到了大卫·韦斯特曾经在步行者队效力，他以一手16尺的毫无破绽的中投、强硬的挡拆和护框、加之敏锐的球场嗅觉、同时还有高圆圆在东决给热火内线造成了巨大打击。'

The chatbot will have memory so we can have follow up conversations.

In [7]:
c.invoke("文档中有多少段落来自“写作”？")

2024-03-13 16:38:14.922 | INFO     | my_notion_companion.conversational_rag:invoke:68 - Sending query to conversatonal RAG: 文档中有多少段落来自“写作”？
Llama.generate: prefix-match hit

llama_print_timings:        load time =    8864.86 ms
llama_print_timings:      sample time =       3.94 ms /    44 runs   (    0.09 ms per token, 11161.85 tokens per second)
llama_print_timings: prompt eval time =     222.82 ms /    32 tokens (    6.96 ms per token,   143.62 tokens per second)
llama_print_timings:        eval time =    1743.22 ms /    43 runs   (   40.54 ms per token,    24.67 tokens per second)
llama_print_timings:       total time =    2042.67 ms /    75 tokens
2024-03-13 16:38:16.973 | INFO     | my_notion_companion.conversational_rag:invoke:84 - Received response: 只有文档1 中有来自 "写作" 的段落，所以不能确定整个资料库中有多少段落来自 "写作"。


'只有文档1 中有来自 "写作" 的段落，所以不能确定整个资料库中有多少段落来自 "写作"。'

The chatbot also has information from the metadata of the document.

In [8]:
c.invoke("这些文档创建的时间是什么时候？")

2024-03-13 16:38:16.997 | INFO     | my_notion_companion.conversational_rag:invoke:68 - Sending query to conversatonal RAG: 这些文档创建的时间是什么时候？
Llama.generate: prefix-match hit

llama_print_timings:        load time =    8864.86 ms
llama_print_timings:      sample time =       5.33 ms /    61 runs   (    0.09 ms per token, 11442.51 tokens per second)
llama_print_timings: prompt eval time =     197.06 ms /    32 tokens (    6.16 ms per token,   162.39 tokens per second)
llama_print_timings:        eval time =    2453.89 ms /    60 runs   (   40.90 ms per token,    24.45 tokens per second)
llama_print_timings:       total time =    2753.99 ms /    92 tokens
2024-03-13 16:38:19.756 | INFO     | my_notion_companion.conversational_rag:invoke:84 - Received response: 文档2 中提到了旅行时间范围为 2020 年 8 月至 2020 年 12 月之间。 不过，没有提供其他文档的创建时间。


'文档2 中提到了旅行时间范围为 2020 年 8 月至 2020 年 12 月之间。 不过，没有提供其他文档的创建时间。'